In [3]:
# Importing modules
import pandas as pd
import os

os.chdir('..')

In [8]:
# Read data
Tweets = pd.read_csv('C:/Users/De/Downloads/Python Files/IM/Beauty_Tweets_Cleaned.csv')

# Print head
Tweets.head()

,SNO,Contents
0,56509,Your favourite perfume can change slightly yea...
1,56510,I m giving away Dior s One Essential Skin Boos...
2,56511,Big congrats to We drew her name this mornin...
3,56512,Do have one or two makeup shades or products y...
4,56513,Just spotted a close up on Cityline of looki...


In [15]:
import re
# Convert the titles to lowercase
Tweets=Tweets.astype(str)
Tweets['Contents_processed'] = Tweets['Contents'].map(lambda x: x.lower())
# Print head
Tweets.head()

,SNO,Contents,Contents_processed
0,56509,Your favourite perfume can change slightly yea...,your favourite perfume can change slightly yea...
1,56510,I m giving away Dior s One Essential Skin Boos...,i m giving away dior s one essential skin boos...
2,56511,Big congrats to We drew her name this mornin...,big congrats to we drew her name this mornin...
3,56512,Do have one or two makeup shades or products y...,do have one or two makeup shades or products y...
4,56513,Just spotted a close up on Cityline of looki...,just spotted a close up on cityline of looki...


In [16]:
# sample only 100 Tweets
Tweets = Tweets.sample(100)


In [18]:
pip install -U gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.24
    Uninstalling Cython-0.29.24:
      Successfully uninstalled Cython-0.29.24
Note: you may need to restart the kernel to use updated packages.


In [19]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = Tweets.Contents_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['changing', 'it', 'up', 'can', 'make', 'all', 'the', 'difference', 'giving', 'our', 'bodies', 'little', 'boost', 'and', 'helping', 'us', 'to', 'look', 'and', 'feel', 'our', 'best', 'little', 'changes', 'that', 'will', 'make', 'big', 'difference', 'to']


In [20]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [21]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\De\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [22]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
pip install -U spacy

  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uninstalled smart-open-6.0.0
Note: you may need to restart the kernel to use updated packages.


In [25]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [26]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['change', 'make', 'difference', 'give', 'body', 'little', 'boost', 'help', 'look', 'feel', 'good', 'little', 'change', 'make', 'big', 'difference', 'beauty', 'regime']


In [27]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 2), (13, 1)]


In [28]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [29]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"new" + 0.024*"eye" + 0.013*"good" + 0.013*"get" + 0.013*"love" + '
  '0.013*"gift" + 0.013*"life" + 0.013*"cute" + 0.013*"honestly" + '
  '0.013*"strong"'),
 (1,
  '0.049*"review" + 0.018*"find" + 0.018*"facial" + 0.018*"luxe" + '
  '0.018*"enhancer" + 0.018*"ty" + 0.018*"week" + 0.018*"rt" + 0.018*"cream" + '
  '0.018*"blog"'),
 (2,
  '0.017*"gift" + 0.017*"get" + 0.017*"ad" + 0.017*"beauty" + 0.017*"towel" + '
  '0.009*"easy" + 0.009*"know" + 0.009*"say" + 0.009*"always" + 0.009*"give"'),
 (3,
  '0.023*"need" + 0.023*"twitter" + 0.012*"long" + 0.012*"keep" + '
  '0.012*"bottle" + 0.012*"back" + 0.012*"linger" + 0.012*"probably" + '
  '0.012*"care" + 0.012*"bathroom"'),
 (4,
  '0.023*"enter" + 0.017*"way" + 0.012*"love" + 0.012*"try" + 0.012*"boundary" '
  '+ 0.012*"push" + 0.012*"good" + 0.012*"brand" + 0.012*"blush" + '
  '0.012*"kind"'),
 (5,
  '0.020*"skin" + 0.020*"see" + 0.020*"winner" + 0.010*"always" + '
  '0.010*"decide" + 0.010*"swatch" + 0.010*"go" + 0.010*"v

In [30]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  nan


C:\Users\De\anaconda3\lib\site-packages\gensim\topic_coherence\direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in double_scalars
  m_lr_i = np.log(numerator / denominator)
C:\Users\De\anaconda3\lib\site-packages\gensim\topic_coherence\indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in double_scalars
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))


In [39]:
#Copied this from https://github.com/RaRe-Technologies/gensim/issues/3040
import logging

import numpy as np
from gensim.topic_coherence import direct_confirmation_measure

log = logging.getLogger(__name__)

ADD_VALUE = 1


def custom_log_ratio_measure(segmented_topics, accumulator, normalize=False, with_std=False, with_support=False):
    topic_coherences = []
    num_docs = float(accumulator.num_docs)
    for s_i in segmented_topics:
        segment_sims = []
        for w_prime, w_star in s_i:
            w_prime_count = accumulator[w_prime]
            w_star_count = accumulator[w_star]
            co_occur_count = accumulator[w_prime, w_star]

            if normalize:
                # For normalized log ratio measure
                numerator = custom_log_ratio_measure([[(w_prime, w_star)]], accumulator)[0]
                co_doc_prob = co_occur_count / num_docs
                m_lr_i = numerator / (-np.log(co_doc_prob + direct_confirmation_measure.EPSILON))
            else:
                # For log ratio measure without normalization
                ### _custom: Added the following 6 lines, to prevent a division by zero error.
                if w_star_count == 0:
                    log.info(f"w_star_count of {w_star} == 0. Adding {ADD_VALUE} to the count to prevent error. ")
                    w_star_count += ADD_VALUE
                if w_prime_count == 0:
                    log.info(f"w_prime_count of {w_prime} == 0. Adding {ADD_VALUE} to the count to prevent error. ")
                    w_prime_count += ADD_VALUE
                numerator = (co_occur_count / num_docs) + direct_confirmation_measure.EPSILON
                denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
                m_lr_i = np.log(numerator / denominator)

            segment_sims.append(m_lr_i)

        topic_coherences.append(direct_confirmation_measure.aggregate_segment_sims(segment_sims, with_std, with_support))

    return topic_coherences

In [41]:
from gensim.topic_coherence import direct_confirmation_measure
from my_custom_module import custom_log_ratio_measure

direct_confirmation_measure.log_ratio_measure = custom_log_ratio_measure

ModuleNotFoundError: No module named 'custom_log_ratio_measure'